In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# should import EXIOBASE database before
# and also set biosphere, LCIA methods
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 2 object(s):
	EXIOBASE 3.3.17 hybrid
	biosphere3

In [4]:
len(bw.methods)

850

In [5]:
eidb = bw.Database('EXIOBASE 3.3.17 hybrid')
len(eidb)

7872

In [6]:
random_act = eidb.random()
random_act.as_dict()

{'location': 'SI',
 'name': 'Activities auxiliary to financial intermediation',
 'code 1': 'i67',
 'code 2': 'A_FAUX',
 'type': 'process',
 'format': 'Exiobase 3.3.17 hybrid mrio_common_metadata tidy datapackage',
 'key': ('EXIOBASE 3.3.17 hybrid',
  'SI-Activities auxiliary to financial intermediation (67)'),
 'unit': 'Meuro',
 'database': 'EXIOBASE 3.3.17 hybrid',
 'code': 'SI-Activities auxiliary to financial intermediation (67)'}

In [7]:
ipcc2013 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [8]:
lca = bw.LCA({random_act:1}, ipcc2013)
lca.lci()
lca.lcia()
lca.score

135415.07358406176

In [9]:
len(lca.biosphere_dict) # number of bio flows

58

In [10]:
# get the information of all activities like name, location
act_dict = {}
act_id = [] # activity id is the number from the lca.activity_dict, to represent the activity
act_name = []
act_loc = []
act_type = []
act_unit = []
act_code = []
for obj in eidb:
    act_id.append(lca.activity_dict[obj['key']])
    act_name.append(obj['name'])
    act_loc.append(obj['location'])
    act_type.append(obj['type'])
    act_unit.append(obj['unit'])
    act_code.append(obj['code'])
len(act_id),len(act_name),len(act_loc),len(act_type),len(act_unit),len(act_code) # check the number of each item

(7872, 7872, 7872, 7872, 7872, 7872)

In [11]:
# make the information as a date frame
act_dict['activity'] = act_id
act_dict['name'] = act_name
act_dict['location'] = act_loc
act_dict['type'] = act_type
act_dict['unit'] = act_unit
act_dict['code'] = act_code
df = pd.DataFrame(act_dict)
df.head()

,activity,name,location,type,unit,code
0,1462,"Composting of paper and wood, incl. land appli...",CZ,process,ton,"CZ-Composting of paper and wood, incl. land ap..."
1,6202,Research and development,ES,process,Meuro,ES-Research and development (73)
2,2162,Mining of other non-ferrous metal ores and con...,DE,process,ton,DE-Mining of other non-ferrous metal ores and ...
3,5308,N-fertiliser,RU,process,ton,RU-N-fertiliser
4,328,Cultivation of paddy rice,BE,process,ton,BE-Cultivation of paddy rice


In [12]:
df.to_csv('exiobase_info.csv')# save file

In [13]:
bio = bw.Database('biosphere3')
random_bio = bio.random()
random_bio.as_dict()

{'categories': ('soil', 'agricultural'),
 'code': '91d68678-7ed7-417a-86a7-a486c7b8a973',
 'name': 'Carfentrazone-ethyl',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission'}

In [14]:
# get reversed lca.biosphere_dict
rev_bio_dict = {value:key for key, value in lca.biosphere_dict.items()}
len(rev_bio_dict)

58

In [15]:
# get the information of bio flows
bio_dict = {}
bio_id = [] # a number from lca.biosphere_dict to represent the bio flow
bio_category = []
bio_name = []
bio_unit = []
bio_type = []
for obj in bio:
    for n in range(58):
        if obj.key == rev_bio_dict[n]:
            bio_id.append(n)
            bio_name.append(obj['name'])
            bio_category.append(obj['categories'])
            bio_unit.append(obj['unit'])
            bio_type.append(obj['type'])
            break
len(bio_id),len(bio_category),len(bio_name),len(bio_unit),len(bio_type) # check the number of each item

(58, 58, 58, 58, 58)

In [16]:
# make the info as a dataframe
bio_dict['bio_id'] = bio_id
bio_dict['name'] = bio_name
bio_dict['category'] = bio_category
bio_dict['unit'] = bio_unit
bio_dict['type'] = bio_type
df_bio = pd.DataFrame(bio_dict)
df_bio.head()

,bio_id,name,category,unit,type
0,18,"Carbon dioxide, non-fossil","(air,)",kilogram,emission
1,44,"Zinc, in ground","(natural resource, in ground)",kilogram,natural resource
2,13,"Sand, unspecified, in ground","(natural resource, in ground)",kilogram,natural resource
3,25,"PAH, polycyclic aromatic hydrocarbons","(air,)",kilogram,emission
4,0,"Aluminium, 24% in bauxite, 11% in crude ore, i...","(natural resource, in ground)",kilogram,natural resource


In [17]:
df_bio.to_csv('exiobase_bioflows_info.csv')#save the file

In [28]:
vectors = {}

In [29]:
# get the lca.inventory scores for each bio flow of activities
from pyprind import prog_bar
for ds in prog_bar(eidb):
    key = lca.activity_dict[ds.key]
    lca.redo_lcia({ds: 1})
    #vectors[key] = get_biosphere_vector(lca)
    vectors[key] = np.ravel((lca.inventory).sum(axis=1))

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:17:59


In [31]:
# make the stored scores as a dataframe
df_data = pd.DataFrame(vectors)
df_data = df_data.T
df_data.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
1999,0.000184,0.004662,0.032859,0.005780,0.000330,0.108512,1.744283,0.000660,0.045615,0.053855,...,0.004624,0.004662,0.000007,0.000007,0.004010,1.150218e-09,0.000002,9.621835e-09,0.000041,8.978172e-07
572,1.383679,-0.260024,-33.362579,-2.966688,-0.839345,-157.809731,-1.179979,-1.678691,-3.508513,101.019461,...,-1.281058,-0.260024,-0.000535,-0.000535,23.469543,1.204124e-06,0.044275,1.708505e-05,-907.901303,7.504524e-03
7562,0.367302,0.031664,103.892944,21.123048,0.711423,256.885736,0.987476,1.422847,76.764317,149.188354,...,8.855054,0.031664,0.000151,0.000151,10.124644,2.725705e-06,0.005653,2.780822e-05,0.092272,6.158178e-04
2817,0.481988,0.148265,356.065851,37.134584,2.524170,71.966726,2.981640,5.048339,203.011663,96.310567,...,3.219319,0.148265,0.000437,0.000437,16.562970,1.948675e-06,0.005832,1.125220e-05,0.140924,5.235760e-04
4526,1.957198,0.776880,614.403005,78.025672,21.852799,1580.959955,181.022078,43.705598,3189.702273,896.535484,...,21.094571,0.776880,0.001578,0.001578,50.128792,8.205923e-06,0.039118,9.150129e-05,0.731665,6.769858e-03


In [32]:
df_data.to_csv('exiobase_inventory_scores.csv') # save the file